# 환경 준비

In [ ]:
# konlpy 설치하기 ## https://data1000.tistory.com/33
!pip3 install jpype1==0.7.0
!pip3 install konlpy

In [ ]:
# konlpy를 실행하기
import konlpy

In [ ]:
## pandas을 실행하기, 단 실행 별명을 pd로. pandas 6자 pd 2자
import pandas as pd

# 데이터 불러오기

In [ ]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## 문서 ID로 실제 파일 불러오기
#  https://drive.google.com/file/d/1NfhNtspQf5oLW4B9XBYmXVLNQEWKTAJJ/view?usp=sharing



rawdata_downloaded = drive.CreateFile({'id': '1NfhNtspQf5oLW4B9XBYmXVLNQEWKTAJJ'})
rawdata_downloaded.GetContentFile('rawdata.csv')

In [ ]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('rawdata.csv')    # utf-8로 저장해서 다시 시도

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,id,author,text,time,votes,heart
0,0,Lector Kreuzritter Kleinbittersdorf,즉시 중국인 입국 금지하지 않으면 멸종한다.\n\n중공 생물학 전쟁연구소의 유출로 ...,2 年前,0,False
1,1,Jay Z,탈출해서 디즈니랜드 돌아다니는 클라쓰..,2 年前,0,True
2,2,Jiyong Kim,6일만에 병원을 짓는다라.. 우리나라 현대건설과 쌍용건설은 돈만 받고 노는거였네? ...,2 年前,0,False
3,3,쭈,그냥 무조건 박쥐나 청결 문제만은 아닌 것 같아요 ..그렇게 따지면 인도나 몇몇 동...,2 年前 (已編輯),0,False
4,4,IQ 175 음악,"New Corona virus treatment / prevention music,...",2 年前,0,False
...,...,...,...,...,...,...
381,389,britemiso,꼭 보세요 끔찍한 중국상황,2 年前,1,False
382,390,미췬텐션,jonghwan Seo 중국만 민폐일까요??\n\n핵오염수 바다에 꾸준히 버리는 일...,2 年前,3,False
383,391,미췬텐션,"a Dog that barks when finding CCP chinese,蟑螂 ...",2 年前,0,False
384,392,이인숙,살아남은게 기적이군요,2 年前,4,False


# 형태소 분석

In [ ]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# "원본데이터"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터.iterrows(): # 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    # 존경하는 국민 여러분. 오늘 저는 대한민국의 제16대 대통령에 취임하기 위해 이...
    분류 = row[1] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [ ]:
# "형태소" 변수의 저장 내용 확인
형태소

[('9:47', 'Number'),
 ('너무', 'Adverb'),
 ('충격', 'Noun'),
 ('적', 'Suffix'),
 ('인', 'Josa'),
 ('결말', 'Noun'),
 ('이네', 'Josa'),
 ('요', 'Noun')]

In [ ]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,9:47,Number,Erebi,1
1,너무,Adverb,Erebi,1
2,충격,Noun,Erebi,1
3,적,Suffix,Erebi,1
4,인,Josa,Erebi,1
5,결말,Noun,Erebi,1
6,이네,Josa,Erebi,1
7,요,Noun,Erebi,1


In [ ]:
# "저장공간" 변수의 저장 내용 확인
저장공간

In [ ]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,즉시,Noun,Lector Kreuzritter Kleinbittersdorf,1
1,중국인,Noun,Lector Kreuzritter Kleinbittersdorf,1
2,입국,Noun,Lector Kreuzritter Kleinbittersdorf,1
3,금지,Noun,Lector Kreuzritter Kleinbittersdorf,1
4,하지,Verb,Lector Kreuzritter Kleinbittersdorf,1
...,...,...,...,...
3,적,Suffix,Erebi,1
4,인,Josa,Erebi,1
5,결말,Noun,Erebi,1
6,이네,Josa,Erebi,1


In [ ]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,즉시,Noun,Lector Kreuzritter Kleinbittersdorf,1
1,중국인,Noun,Lector Kreuzritter Kleinbittersdorf,1
2,입국,Noun,Lector Kreuzritter Kleinbittersdorf,1
3,금지,Noun,Lector Kreuzritter Kleinbittersdorf,1
4,하지,Verb,Lector Kreuzritter Kleinbittersdorf,1
...,...,...,...,...
3,적,Suffix,Erebi,1
4,인,Josa,Erebi,1
5,결말,Noun,Erebi,1
6,이네,Josa,Erebi,1


In [ ]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소                품사         분류          
\n                 Foreign    17 BLUE         1
                              1만시간의 법칙        1
                              : 0             1
                              B찢재명            3
                              Daniel          1
                                             ..
힘들어                Adjective  서원석             1
힘듬                 Adjective  초심              1
힘듭니다               Adjective  Boungmin Lee    1
😡😡😡😡😡              Foreign    Yun Soo         2
🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔  Foreign    Yun Soo         2
Name: 카운트, Length: 8698, dtype: int64

In [ ]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소               품사        분류               
\n                Foreign   17 BLUE         1
                            1만시간의 법칙        1
                            : 0             1
                            B찢재명            3
                            Daniel          1
...                                       ...
힘들어               Adjective 서원석             1
힘듬                Adjective 초심              1
힘듭니다              Adjective Boungmin Lee    1
😡😡😡😡😡             Foreign   Yun Soo         2
🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔🤔 Foreign   Yun Soo         2

[8698 rows x 1 columns]

In [ ]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')